In [1]:
#pip install -e .
#pip install  abbyy_course_cvdl_t2

from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np

import torch

from task1pack.utils.data import HeatmapDataset

/home/avashchilko/abbyy9sem/course_cvdl/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = Path() / '..' / '..' / 'data' / 'midv500_compressed'
#DATASET_PATH = Path() / '..' / '..' / '..' / '..' / '..' / '..' / 'Downloads' / 'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

In [3]:
# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

In [4]:
from torchvision.transforms import Resize, Compose, ToTensor

IMAGE_SIZE = [512, 512]
HEATMAP_SIZE = [128, 128]

transforms = Compose([
    ToTensor(),
    Resize(IMAGE_SIZE),
])

In [5]:
train_dataset = HeatmapDataset(data_packs=data_packs, split='train', transforms=transforms, output_size=HEATMAP_SIZE)
test_dataset = HeatmapDataset(data_packs=data_packs, split='test', transforms=transforms, output_size=HEATMAP_SIZE)

len(train_dataset), len(test_dataset)

(10750, 4250)

In [6]:
train_dataset[0][0].shape, train_dataset[0][1].shape

(torch.Size([3, 512, 512]), torch.Size([4, 2]))

In [7]:
torch.manual_seed(42)

In [8]:
from task1pack.models.hrnet import get_pose_net
from task1pack.models.hrnet_config import cfg

In [9]:
cfg['MODEL']['IMAGE_SIZE'] = IMAGE_SIZE
cfg['MODEL']['HEATMAP_SIZE'] = HEATMAP_SIZE

hrnet = get_pose_net(is_train=True, cfg=cfg)

In [10]:
from task1pack.utils.train import train_model, show_train_plots, train_old, train_loop
from torch.nn import MSELoss
import wandb

In [11]:
model_name = 'HRNet'

device = 'cuda:1'
criterion = MSELoss()

num_epochs = 100

wandb.init(
    project='ocr task 1',
    name='{} {} epochs with lr={} new train'.format(
        model_name, 
        num_epochs, 
        0.001,),
    config={
        "batch_size": 16,
        "architecture": model_name,
        "dataset": "MIDV-500",
        "criterion": "MSELoss",
        "optimizer": "Adam",
        "image_size": IMAGE_SIZE,
        "heatmap_size": HEATMAP_SIZE,    
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin


In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
train_losses, test_losses, trained_model = train_loop(
    epochs=num_epochs,
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    model=hrnet, 
    criterion=criterion, 
    device=device,
    wandb_instance=wandb,
    step_size=35,
)

/home/avashchilko/abbyy9sem/course_cvdl/.venv/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Initital test loss: 0.1179849989414215
Epoch 1:
training
evaluating
train loss: 0.12619503806496776
Epoch 2:
training
evaluating
train loss: 0.06315144538810087
Epoch 3:
training
evaluating
train loss: 0.03188177799831989
Epoch 4:
training
evaluating
train loss: 0.029259758943388628
Epoch 5:
training
evaluating
train loss: 0.03064972708876743
Epoch 6:
training
evaluating
train loss: 0.029986520093540813
Epoch 7:
training
evaluating
train loss: 0.03142129924789418
Epoch 8:
training
evaluating
train loss: 0.028698820101140544
Epoch 9:
training
evaluating
train loss: 0.029037454853917278
Epoch 10:
training
evaluating
test loss: 0.0163924969301066
train loss: 0.027585797032124774
Epoch 11:
training
evaluating
train loss: 0.027476341903382957
Epoch 12:
training
evaluating
train loss: 0.027490562621244163
Epoch 13:
training
evaluating
train loss: 0.02775467951075975
Epoch 14:
training
evaluating
train loss: 0.026987099145214227
Epoch 15:
training
evaluating
train loss: 0.0264021323576223
Epo

In [ ]:
show_train_plots(train_losses, test_losses, 'HRNet')

In [ ]:
from course_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

results_dict = {}

trained_model.eval()

with torch.no_grad():
    for i, (x, _) in tqdm(enumerate(test_dataset)):
        result = trained_model(x[None, ...].to(device))[0]
        key = test_dataset.get_key(i)
        results_dict[key] = result.cpu().tolist()

dump_results_dict(results_dict, Path() / 'pred1.json')

acc_95 = measure_crop_accuracy(
    Path() / 'pred1.json',
    Path() / 'gt.json'
)

acc_80 = measure_crop_accuracy(
    Path() / 'pred1.json',
    Path() / 'gt.json',
    iou_thr=0.8
)

wandb.run.summary['acc@95'] = acc_95
wandb.run.summary['acc@80'] = acc_80

torch.save(trained_model.state_dict(), Path() / 'model1.pth')
wandb.save(str(Path() / 'model1.pth'))
wandb.save(str(Path() / 'pred1.json'))

wandb.finish()

print(acc_95)